In [52]:
import sys
import pandas as pd
import numpy as np

# utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

# decision tree
from sklearn.tree import DecisionTreeRegressor

# svm
from sklearn.svm import SVR
from sklearn import preprocessing

# NN
from sklearn.neural_network import MLPRegressor

# so values can be viewed as scrollable element
np.set_printoptions(threshold=sys.maxsize)


In [53]:
def CalcTotalProb(X_data_og):
  X_data = X_data_og.copy()
  # Calculate Probabilities
  total_crashes = X_data.shape[0]
  total_prob = []

  hour_prob = {}
  day_prob = {}
  month_prob = {}

  # probability for each hour
  for hour in range(24):
      occurrences_in_hour = X_data['Hour'].value_counts()[hour]
      hour_prob[hour] = occurrences_in_hour/total_crashes

  for day in range(1,8):
      occurrences_in_day = X_data['Day'].value_counts()[day]
      day_prob[day] = occurrences_in_day/total_crashes

  months = X_data['Month'].unique()
  for month in months:
      occurrences_in_month = X_data['Month'].value_counts()[month]
      month_prob[month] = occurrences_in_month/total_crashes
      
  # when calculating the total probability, the value gets extremely small due to multiplying
      # probabilities. To reduce the effect of this we'll use log probabilities which shouldn't
      # affect effectiveness of our product since we're simply comparing probabilities
  def CalculateTotalProb(row):
      probabilities = [hour_prob[row['Hour']],
                      day_prob[row['Day']],
                      month_prob[row['Month']]]
      log_probabilities = np.log(probabilities)
      log_combined_probability  = np.sum(log_probabilities)
      return log_combined_probability * -1

  X_data['total_prob'] = X_data.apply(CalculateTotalProb, axis=1)

  return X_data['total_prob']

# Generating total_prob

Total probability is the probability of a given combination of travel details appearing in the crash database. The total_prob is therefore calculated manually for the training, testing, and validation splits to avoid data leakage. 

In [69]:
data_filepath = ".\\modified_data\\cleaned_data.csv"
df = pd.read_csv(data_filepath)

# splitting data: We will use a 80-20 split
X_train, X_test = train_test_split(df, test_size=0.2, random_state=23)

# Y_train is the probability of each value for its iven 
Y_train = CalcTotalProb(X_train)
Y_test = CalcTotalProb(X_test)

train, Xtest = train_test_split(df, test_size=0.3, random_state=23)
Xtrain, Xval = train_test_split(train, test_size=0.25, random_state=23)

Ytrain = CalcTotalProb(Xtrain)
Ytest = CalcTotalProb(Xtest)
Yval = CalcTotalProb(Xval)

Note: Latitude and Longitude are not part of total_prob

# Creating Model

We determined through EDA that a simple model will not be sufficient to predict the probability type. We will instead use more complicated models (Decision trees, SVM). If those models don't work, we will increase complexity even more to random forest and neural networks.

In [50]:
# create decision tree classifier
clf = DecisionTreeRegressor(random_state=23)

# train classifier
clf.fit(X_train, Y_train)

# Make predictions on the test data
y_train_pred = clf.predict(X_train)
y_pred = clf.predict(X_test)

# Calculate MSE and R2
print("Training MSE: %.2f" % (mean_squared_error(y_train_pred, Y_train)))
print("Testing MSE: %.2f" % (mean_squared_error(y_pred, Y_test)))

r2 = r2_score(Y_train, y_train_pred)
print('Training R2: %.2f' % r2)
r2 = r2_score(Y_test, y_pred)
print('Test R2: %.2f' % r2)

Training MSE: 0.00
Testing MSE: 0.00
Training R2: 1.00
Test R2: 0.99


These errors seem too good to be true so we will use the tougher "train, test, validate" split to check

In [55]:
# create decision tree classifier
clf = DecisionTreeRegressor(random_state=23)

# train classifier
clf.fit(Xtrain, Ytrain)

# Make predictions on the test data
y_train_pred = clf.predict(Xtrain)
y_test_pred = clf.predict(Xtest)
y_val_pred = clf.predict(Xval)

# Calculate MSE and R2
print("Training MSE: %.2f" % (mean_squared_error(y_train_pred, Ytrain)))
print("Testing MSE: %.2f" % (mean_squared_error(y_test_pred, Ytest)))
print("Validation MSE: %.2f" % (mean_squared_error(y_val_pred, Yval)))

r2 = r2_score(Ytrain, y_train_pred)
print('Training R2: %.2f' % r2)
r2 = r2_score(Ytest, y_test_pred)
print('Test R2: %.2f' % r2)
r2 = r2_score(Yval, y_val_pred)
print('Validation R2: %.2f' % r2)

Training MSE: 0.00
Testing MSE: 0.00
Validation MSE: 0.00
Training R2: 1.00
Test R2: 0.99
Validation R2: 0.99


The errors remained the same and they're equally good for both training, testing, and validation so there's no overfitting. We will stop here for trees since we've achieved nearly perfect accuracy.

# SVM Model

We want to one-hot encode categorical data and try different kernels. We'll keep other parameters at their defaults unless this model seems more promising than the others.

In [27]:
cats = ['Month', 'Day', 'Hour']

df_svm = df.copy()

df_svm = pd.get_dummies(df_svm, columns=cats)
df_svm = df_svm.astype(float)

# split data
svm_train, svm_test = train_test_split(df_svm, test_size=0.2)
X_svm_train, y_svm_train = svm_train.drop(columns=['total_prob']), svm_train['total_prob']
X_svm_test, y_svm_test = svm_test.drop(columns=['total_prob']), svm_test['total_prob']

In [14]:
svc_li = SVR(kernel='linear')

scaler = preprocessing.StandardScaler()
scaler.fit(X_svm_train)

Z_svm_train = scaler.transform(X_svm_train)
Z_svm_test = scaler.transform(X_svm_test)

svc_li.fit(Z_svm_train, np.asarray(y_svm_train))

print('Linear Kernel')
# Calculate MSE and R2
y_train_pred = svc_li.predict(Z_svm_train)
y_test_pred = svc_li.predict(Z_svm_test)
print("Training MSE: %.2f" % (mean_squared_error(y_train_pred, y_svm_train)))
print("Testing MSE: %.2f" % (mean_squared_error(y_test_pred, y_svm_test)))

r2 = r2_score(y_train_pred, y_train_pred)
print('Training R2: %.2f' % r2)
r2 = r2_score(y_test_pred, y_pred)
print('Test R2: %.2f' % r2)

svc_rbf = SVR(kernel='rbf')
svc_rbf.fit(Z_svm_train, np.asarray(y_svm_train))

print('')
print('rbf Kernel')
# Calculate MSE and R2
y_train_pred = svc_rbf.predict(Z_svm_train)
y_test_pred = svc_rbf.predict(Z_svm_test)
print("Training MSE: %.2f" % (mean_squared_error(y_train_pred, y_svm_train)))
print("Testing MSE: %.2f" % (mean_squared_error(y_test_pred, y_svm_test)))

r2 = r2_score(y_train_pred, y_train_pred)
print('Training R2: %.2f' % r2)
r2 = r2_score(y_test_pred, y_pred)
print('Test R2: %.2f' % r2)

Linear Kernel
Training MSE: 0.01
Testing MSE: 0.01
Training R2: 1.00
Test R2: -1.32

rbf Kernel
Training MSE: 0.00
Testing MSE: 0.00
Training R2: 1.00
Test R2: -1.02


**SVM is a poor model for this use case** as is evident by its negative R2 score. It does well in training for both kernels but loses all its performance when operating on the test split.

# Neural Network

In [62]:
# perform one hot encoding
cats = ['Month', 'Day', 'Hour']
X_train = pd.get_dummies(X_train, columns=cats)
X_train = X_train.astype(float)
X_test = pd.get_dummies(X_test, columns=cats)
X_test = X_test.astype(float)
# print(X)

# normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_rescaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(data = X_train_rescaled, columns = X_train.columns)

X_test_rescaled = scaler.fit_transform(X_test)
X_test = pd.DataFrame(data = X_test_rescaled, columns = X_test.columns)

In [63]:
num_nodes = (23, 17, 13)
learning_rate = 0.4
epochs = 500

mlp = MLPRegressor(solver='sgd', random_state = 0, activation = 'logistic', learning_rate_init = learning_rate, batch_size = 100, hidden_layer_sizes = num_nodes, max_iter = epochs)
mlp.fit(X_train, Y_train)

MLPRegressor(activation='logistic', batch_size=100,
             hidden_layer_sizes=(23, 17, 13), learning_rate_init=0.4,
             max_iter=500, random_state=0, solver='sgd')

In [64]:
y_train_pred =  mlp.predict(X_train)
y_test_pred = mlp.predict(X_test)

print("Training MSE: %.2f" % (mean_squared_error(y_train_pred, Y_train)))
print("Testing MSE: %.2f" % (mean_squared_error(y_test_pred, Y_test)))

r2 = r2_score(y_train_pred, Y_train)
print('Training R2: %.2f' % r2)
r2 = r2_score(y_test_pred, Y_test)
print('Test R2: %.2f' % r2)


Training MSE: 0.34
Testing MSE: 0.35
Training R2: -48263799516023113679466659840.00
Test R2: -48766868715701755468434112512.00


Very poor performance, let's try swapping the solver

In [65]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)
mlp.fit(X_train, Y_train)

MLPRegressor(hidden_layer_sizes=(100, 50), random_state=42)

In [66]:
# training errors
y_pred = mlp.predict(X_train)
print("Mean Squared Error:", mean_squared_error(Y_train, y_pred))
print('Test R2: %.2f' % r2_score(y_pred, Y_train))

# testing errors
y_pred = mlp.predict(X_test)
print("Mean Squared Error:", mean_squared_error(Y_test, y_pred))
print('Test R2: %.2f' % r2_score(y_pred, Y_test))

Mean Squared Error: 0.00044394685388621024
Test R2: 1.00
Mean Squared Error: 0.030363943525519414
Test R2: 0.91


This is an extremely good prediction rate and seems too good to be true. We will verify by using a train, test, validation split to further check

In [70]:
# perform one hot encoding
cats = ['Month', 'Day', 'Hour']
Xtrain = pd.get_dummies(Xtrain, columns=cats)
Xtrain = Xtrain.astype(float)
Xtest = pd.get_dummies(Xtest, columns=cats)
Xtest = Xtest.astype(float)
Xval = pd.get_dummies(Xval, columns=cats)
Xval = Xval.astype(float)
# print(X)

# normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
Xtrain_rescaled = scaler.fit_transform(Xtrain)
Xtrain = pd.DataFrame(data = Xtrain_rescaled, columns = Xtrain.columns)

Xtest_rescaled = scaler.fit_transform(Xtest)
Xtest = pd.DataFrame(data = Xtest_rescaled, columns = Xtest.columns)

Xval_rescaled = scaler.fit_transform(Xval)
Xval = pd.DataFrame(data = Xval_rescaled, columns = Xval.columns)

In [71]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)
mlp.fit(Xtrain, Ytrain)

# training errors
y_pred = mlp.predict(Xtrain)
print("Mean Squared Error:", mean_squared_error(Ytrain, y_pred))
print('Test R2: %.2f' % r2_score(y_pred, Ytrain))

# testing errors
y_pred = mlp.predict(Xtest)
print("Mean Squared Error:", mean_squared_error(Ytest, y_pred))
print('Test R2: %.2f' % r2_score(y_pred, Ytest))

# validation errors
y_pred = mlp.predict(Xval)
print("Mean Squared Error:", mean_squared_error(Yval, y_pred))
print('Test R2: %.2f' % r2_score(y_pred, Yval))

Mean Squared Error: 0.0007115272536880937
Test R2: 1.00
Mean Squared Error: 0.04937255760290731
Test R2: 0.86
Mean Squared Error: 0.010193607408828922
Test R2: 0.97


Near-perfect error holds for validation set too so we can trust that this model performs well. We know it is not overfitting because the testing R2 is just as high as the training value.